<a href="https://colab.research.google.com/github/cenard/Finance_hobby/blob/main/BTC_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import the libraries used in this project
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pycaret.regression import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [45]:
from google.colab import files
files.upload()
# if you have an issue uploading your file in Firefox, try switching off the Enhanced Protection for the site, 
# which is accessible via the little "security shield" to the left of the URL, that works great for me


Saving BTC_Binance.csv to BTC_Binance.csv


{'BTC_Binance.csv': b'date,open,high,low,close,Volume USDT\n2022-06-29 00:00:00,20281.28,20320.99,20152,20315.05,28588278.4403553\n2022-06-28 00:00:00,20742.57,21212.1,20202.01,20281.29,1321862977.93155\n2022-06-27 00:00:00,21038.08,21539.85,20510,20742.56,1354369225.84047\n2022-06-26 00:00:00,21491.18,21888,20964.73,21038.07,1139258367.16292\n2022-06-25 00:00:00,21237.68,21614.5,20906.62,21491.19,1093873137.97475\n2022-06-24 00:00:00,21110.12,21558.41,20736.72,21237.69,1635129191.17025\n2022-06-23 00:00:00,19988,21233,19890.07,21110.13,1710694866.45724\n2022-06-22 00:00:00,20723.51,20900,19770.51,19987.99,1872204589.23758\n2022-06-21 00:00:00,20573.9,21723,20348.4,20723.52,2198596137.34328\n2022-06-20 00:00:00,20574,21090,19637.03,20573.89,2220396381.74265\n2022-06-19 00:00:00,18970.79,20815.95,17960.41,20574,2487561781.53599\n2022-06-18 00:00:00,20468.81,20792.06,17622,18970.79,3726639821.69615\n2022-06-17 00:00:00,20400.6,21365.43,20246.66,20468.81,1796575392.55072\n2022-06-16 00:00

In [46]:
# Read and show the data stored into the 'asset' variable
asset = pd.read_csv("BTC_Binance.csv")
asset

,date,open,high,low,close,Volume USDT
0,2022-06-29 00:00:00,20281.28,20320.99,20152.00,20315.05,2.858828e+07
1,2022-06-28 00:00:00,20742.57,21212.10,20202.01,20281.29,1.321863e+09
2,2022-06-27 00:00:00,21038.08,21539.85,20510.00,20742.56,1.354369e+09
3,2022-06-26 00:00:00,21491.18,21888.00,20964.73,21038.07,1.139258e+09
4,2022-06-25 00:00:00,21237.68,21614.50,20906.62,21491.19,1.093873e+09
...,...,...,...,...,...,...
1774,2017-08-21,4086.29,4119.62,3911.79,4016.00,2.770592e+06
1775,2017-08-20,4139.98,4211.08,4032.62,4086.29,1.915636e+06
1776,2017-08-19,4108.37,4184.69,3850.00,4139.98,1.508239e+06
1777,2017-08-18,4285.08,4371.52,3938.77,4108.37,4.994494e+06


In [49]:
# [VALIDATION BOX - you can go next]
# at first I had a problem with the columns, the DB provider put his URL as column and I had to edit that before continue
# print(asset.columns.tolist())

['close', 'future_price']


In [48]:
# variable for prediction 1 day out into the future
future_days = 1

# create a new column (the target or dependent variable) shifted 'n' units up, in this case n=1
asset['future_price'] = asset[['close']].shift(-future_days)

# choose the column
asset = asset[['close', 'future_price']]
asset

,close,future_price
0,20315.05,20281.29
1,20281.29,20742.56
2,20742.56,21038.07
3,21038.07,21491.19
4,21491.19,21237.69
...,...,...
1774,4016.00,4086.29
1775,4086.29,4139.98
1776,4139.98,4108.37
1777,4108.37,4285.08


In [54]:
# make a copy of the data set
df = asset.copy()

# create the indepent data set
X = np.array(df[df.columns])

# remove the last 'n' rows from the data set where 'n' is the future_days = 1
X = X[:len(asset)-future_days]

# create the dependemt data set (Y) and convert the dataframe to a numpy array
y = np.array(df["future_price"])

# get all of the y values except for the last 'n' rows which is the future_days = 1
y = y[:-future_days]

# split the data into a 85% training data set and a 15% testing data set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 0, shuffle=False)

In [67]:
# get the train data and tranform it to a data frame
train_data = pd.DataFrame(x_train, columns=df.columns)

#Show the first 7 rows of data
train_data

,close,future_price
0,20315.05,20281.29
1,20281.29,20742.56
2,20742.56,21038.07
3,21038.07,21491.19
4,21491.19,21237.69
...,...,...
1506,8462.00,8663.34
1507,8663.34,8679.71
1508,8679.71,8465.94
1509,8465.94,8400.00


In [66]:
# doing the same for the testing case
testing_data = pd.DataFrame(x_test, columns=df.columns)

testing_data

,close,future_price
0,9002.20,9310.00
1,9310.00,9187.56
2,9187.56,9365.00
3,9365.00,9659.01
4,9659.01,9864.00
...,...,...
262,4040.00,4016.00
263,4016.00,4086.29
264,4086.29,4139.98
265,4139.98,4108.37


In [68]:
# initialize the setup [IMPORTANT INFO]
regression_setup = setup(data=train_data, target= 'future_price', session_id=123, use_gpu=True)

,Description,Value
0,session_id,123
1,Target,future_price
2,Original Data,"(1511, 2)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1057, 1)"


In [69]:
# train on all of the models and sort it by the R-squared metric (r2)
# then we store the odel with the best r2 score

best_model = compare_models(sort='r2')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,618.5704,1.265203e+06,1113.4511,0.9963,0.0398,0.0265,0.012
lar,Least Angle Regression,621.2246,1.267483e+06,1114.4977,0.9962,0.0400,0.0270,0.008
lr,Linear Regression,621.2245,1.267483e+06,1114.4976,0.9962,0.0400,0.0270,0.007
omp,Orthogonal Matching Pursuit,621.2246,1.267484e+06,1114.4977,0.9962,0.0400,0.0270,0.008
llar,Lasso Least Angle Regression,625.2077,1.268433e+06,1114.9998,0.9962,0.0403,0.0278,0.007
br,Bayesian Ridge,621.2317,1.267484e+06,1114.4979,0.9962,0.0400,0.0270,0.008
en,Elastic Net,621.2248,1.267484e+06,1114.4977,0.9962,0.0400,0.0270,0.010
ridge,Ridge Regression,621.2247,1.267484e+06,1114.4977,0.9962,0.0400,0.0270,0.007
lasso,Lasso Regression,621.2248,1.267484e+06,1114.4977,0.9962,0.0400,0.0270,0.009
gbr,Gradient Boosting Regressor,676.6196,1.425981e+06,1182.4460,0.9958,0.0441,0.0298,0.083


In [70]:
# create the final model with the highest r2
# showing its prediction metrics on the training data set

model = create_model(best_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,836.2807,1.961139e+06,1400.4066,0.9951,0.0422,0.0295
1,650.1432,1.172889e+06,1083.0001,0.9964,0.0605,0.0292
2,588.1449,9.564971e+05,978.0067,0.9968,0.0371,0.0262
3,540.4395,8.207828e+05,905.9706,0.9976,0.0334,0.0236
4,595.6737,1.576543e+06,1255.6048,0.9949,0.0390,0.0272
5,714.4338,1.768701e+06,1329.9252,0.9949,0.0424,0.0293
6,454.3402,8.426618e+05,917.9661,0.9973,0.0312,0.0208
7,609.9724,1.267845e+06,1125.9864,0.9962,0.0394,0.0278
8,570.1809,1.120576e+06,1058.5727,0.9968,0.0359,0.0242


In [71]:
# evaluate the model
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [73]:
# get the final predictions
unseen_predictions = predict_model(model, data=train_data)

# show the predictions
unseen_predictions

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Huber Regressor,593.1097,1.175683e+06,1084.2891,0.9965,0.0391,0.0259


,close,future_price,Label
0,20315.05,20281.29,20274.883816
1,20281.29,20742.56,20241.193474
2,20742.56,21038.07,20701.489772
3,21038.07,21491.19,20996.374793
4,21491.19,21237.69,21448.535983
...,...,...,...
1506,8462.00,8663.34,8446.891867
1507,8663.34,8679.71,8647.806054
1508,8679.71,8465.94,8664.141562
1509,8465.94,8400.00,8450.823967
